In [9]:
print(train.columns.tolist())
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier

# Load data
train = pd.read_csv("/kaggle/input/binary-classification-with-a-bank-dataset-clone/train.csv")
test = pd.read_csv("/kaggle/input/binary-classification-with-a-bank-dataset-clone/test.csv")

# Target
y = train["y"]

# Drop target from train
train_features = train.drop("y", axis=1)

# Combine train + test for encoding
combined = pd.concat([train_features, test], axis=0)

# One-Hot Encoding
combined_encoded = pd.get_dummies(combined, drop_first=True)

# Split back
X = combined_encoded.iloc[:len(train)]
X_test = combined_encoded.iloc[len(train):]

# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# Model
model = XGBClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric="auc",
    random_state=42
)

# Train
model.fit(X_train, y_train)

# Validation
val_preds = model.predict_proba(X_val)[:, 1]
print("Validation ROC-AUC:", roc_auc_score(y_val, val_preds))

# Predict test
test_preds = model.predict_proba(X_test)[:, 1]

# Submission
submission = pd.DataFrame({
    "id": test["id"],
    "y": test_preds
})

submission.to_csv("submission.csv", index=False)
print("submission.csv created successfully")


['id', 'age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']
Validation ROC-AUC: 0.9618868740759102
submission.csv created successfully
